In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

import os

In [2]:
df = pd.read_csv("data/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
df.describe(percentiles=[0.25, 0.5, 0.6,0.75])

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
60%,6000.40000,1.571569e+07,678.000000,40.000000,6.000000,110138.926000,2.000000,1.00000,1.000000,119710.038000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [6]:
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
#train test split 
X_train, X_test, y_train, y_test = train_test_split(df.drop("Exited", axis=1), df["Exited"], test_size=0.2, random_state=42)

In [ ]:
# Label Encoding for Gender Column
lablecoder = LabelEncoder()
X_train["Gender"] = lablecoder.fit_transform(X_train["Gender"])
X_test["Gender"] = lablecoder.transform(X_test["Gender"])

In [ ]:
# One-Hot Encoding for Geography Column
onecoder = OneHotEncoder(sparse_output=True, dtype=np.int64)
X_train_geo = onecoder.fit_transform(X_train["Geography"].values.reshape(-1, 1))
X_test_geo = onecoder.transform(X_test["Geography"].values.reshape(-1, 1))

In [ ]:
# Combine the encoded features with the original dataset (excluding the original "Geography" column)
x_train_encpd = pd.DataFrame(X_train_geo.toarray(), columns=onecoder.get_feature_names_out(["Geography"]))
pd.concat([X_train.drop("Geography", axis=1).reset_index(drop=True), x_train_encpd], axis=1)

x_test_encpd = pd.DataFrame(X_test_geo.toarray(), columns=onecoder.get_feature_names_out(["Geography"]))
pd.concat([X_test.drop("Geography", axis=1).reset_index(drop=True), x_test_encpd], axis=1)



,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,686,1,32,6,0.00,2,1,1,179093.26,1,0,0
1,632,1,42,4,119624.60,2,1,1,195978.86,0,1,0
2,559,1,24,3,114739.92,1,1,0,85891.02,0,0,1
3,561,0,27,9,135637.00,1,1,0,153080.40,1,0,0
4,517,1,56,9,142147.32,1,0,0,39488.04,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,768,1,54,8,69712.74,1,1,1,69381.05,1,0,0
7996,682,0,58,1,0.00,1,1,1,706.50,1,0,0
7997,735,0,38,1,0.00,3,0,0,92220.12,1,0,0
7998,667,1,43,8,190227.46,1,1,0,97508.04,1,0,0


In [ ]:
# save the processed data and encoders
os.makedirs("artifacts", exist_ok=True)
with open("artifacts/onehotencoder_gender.pkl", "wb") as file:
    pickle.dump(onecoder, file)

with open("artifacts/labelencoder_geography.pkl", "wb") as file:
    pickle.dump(lablecoder, file)

In [24]:
X_train.drop("Geography", axis=1, inplace=True)
X_test.drop("Geography", axis=1, inplace=True)

In [25]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# save the scaler
with open("artifacts/standardscaler.pkl", "wb") as file:
    pickle.dump(scaler, file)